In [1]:
import numpy as np
import matplotlib.pylab as plt
from skimage import io
from skimage import img_as_uint

from phase_opti import phase_registration_optim

import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from stretchablecorr import *
import filetools as ft

ImportError: cannot import name 'phase_registration_optim'

In [101]:
cubes = []
cube, image_names = ft.load_image_sequence('./images/hs2/0p1')
cubes.append(cube)
cube, image_names = ft.load_image_sequence('./images/hs2/5p0')
cubes.append(cube)

Image sequence:
 11 frames
 1392*1024 pixels
 memory size: 119 Mo
 images: hs20p100012.TIF, hs20p100013.TIF, hs20p100014.TIF , ... ,  hs20p100022.TIF
Image sequence:
 11 frames
 1392*1024 pixels
 memory size: 119 Mo
 images: hs25p000190.TIF, hs25p000191.TIF, hs25p000192.TIF , ... ,  hs25p000200.TIF


In [102]:
import itertools as it

In [103]:
window_half_size = 100
xy_center = 200, 500
shifts_opti = []
shifts_sk   = []

for i, j in it.product(range(len(cubes[0])), range(len(cubes[1]))):
    A, xy0 = crop(cubes[0][i], xy_center, window_half_size)
    B, xy0 = crop(cubes[1][j], xy_center, window_half_size)
    
    shift, hess_inv, res = phase_registration_optim(A, B, phase=True)
    shifts_opti.append(shift)

    shift, _, _ = phase_cross_correlation(A, B, upsample_factor=50)
    shifts_sk.append(shift)
    print(i, j, ' '*10, end='\r')

In [104]:
shifts_opti = np.vstack(shifts_opti)
shifts_sk   = np.vstack(shifts_sk)

In [105]:
d0 = shifts_opti.mean(axis=0)
print(d0)
d = np.sqrt(np.sum((shifts_opti - d0)**2, axis=1))
print(d.mean())

[-4.49525185 -2.83917696]
0.16761954659620637


In [106]:
d0 = shifts_sk.mean(axis=0)
print(d0)
d = np.sqrt(np.sum((shifts_sk - d0)**2, axis=1))
print(d.mean())

[-4.40876033 -3.07917355]
0.13865012797864518


In [107]:
shift, hess_inv, res = phase_registration_optim(A, B, phase=True)
print(shift)

shift, _, _ = phase_cross_correlation(A, B, upsample_factor=20)
print(shift)

[-4.84214824 -3.02616978]
[-4.5  -3.25]


In [108]:
hess_inv

array([[2.15757062e-04, 4.11896466e-05],
       [4.11896466e-05, 1.14456258e-04]])

In [100]:
np.sqrt(8.55960862e-05*31*3)

0.08922127558267703

In [99]:
np.std(cubes[0][6]-cubes[0][2])

31.641726603973996